In [41]:
import numpy as np
import pandas as pd
import datetime
import time

Importing the amazon data

In [43]:
start_time = time.time()

import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

rating = getDF('reviews_Digital_Music_5.json.gz')
meta = getDF('meta_Digital_Music.json.gz')

print(time.time() - start_time, "seconds")

32.56266713142395 seconds


printing dimensions

In [44]:
print("\nmetadata dimensions")
print(meta.shape)
print("\nrating dimensions")
print(rating.shape)


metadata dimensions
(279899, 9)

rating dimensions
(64706, 9)


In [45]:
# Restricting title sizes
def auto_truncate_description(val):
    return val[:1024]
def auto_truncate_title(val):
    return val[:255]

In [46]:
rating.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5.0,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5.0,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5.0,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5.0,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4.0,4.5; music to dream to,1200528000,"01 17, 2008"


Only using (user-item-rating-timestamp) for now

In [47]:
df_ratings = rating[["reviewerID", "asin", "overall", "unixReviewTime"]]
df_ratings.unixReviewTime = pd.to_datetime(df_ratings["unixReviewTime"],unit = 's')

#changing column names
df_ratings = df_ratings.rename(columns={'reviewerID': 'user_id', 'asin': 'item_id', 'overall': 'rating', 'unixReviewTime': 'timestamp'})


C:\Users\Nabeel\Anaconda2\envs\py36\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [48]:
df_ratings.head()

,user_id,item_id,rating,timestamp
0,A3EBHHCZO6V2A4,5555991584,5.0,2006-09-12
1,AZPWAXJG9OJXV,5555991584,5.0,2001-06-03
2,A38IRL0X2T4DPF,5555991584,5.0,2003-07-14
3,A22IK3I6U76GX0,5555991584,5.0,2000-05-03
4,A1AISPOIIHTHXX,5555991584,4.0,2008-01-17


Parsing the item_ids to an integer format and taking the top 1000 items only

In [62]:
item_filter = pd.DataFrame(df_ratings['item_id'])
item_filter = item_filter.groupby('item_id').size()
item_filter = item_filter.to_frame().reset_index()
item_filter.columns.values[1] = 'count'
item_filter = item_filter.sort_values('count', ascending=False)


In [63]:
print(item_filter['count'].mean())
print(item_filter['count'].max())
print(item_filter['count'].min())
item_filter.head()

18.135089686098656
272
5


,item_id,count
2432,B0007NFL18,272
2144,B000084T18,271
2050,B00006690F,204
1777,B00004T9UF,202
2025,B00005YW4H,190


In [64]:
item_filter = item_filter.head(n=1000)

In [65]:
print(item_filter['count'].mean())
print(item_filter['count'].max())
print(item_filter['count'].min())
item_filter.head()

42.117
272
18


,item_id,count
2432,B0007NFL18,272
2144,B000084T18,271
2050,B00006690F,204
1777,B00004T9UF,202
2025,B00005YW4H,190


In [66]:
item_filter = item_filter[['item_id']]

In [67]:
df_ratings = pd.merge(left=df_ratings,right=item_filter, left_on='item_id', right_on='item_id')

In [68]:
df_ratings = df_ratings.sort_values("user_id")

In [69]:
df_ratings.user_id = df_ratings.user_id.astype("category")

df_ratings["userid_key"] = df_ratings["user_id"].cat.codes
df_ratings["userid_key"] = df_ratings["userid_key"] + 1


In [70]:
df_ratings.head()

,user_id,item_id,rating,timestamp,userid_key
8318,A08161909WK3HU7UYTMW,B000002H1C,3.0,2013-01-18,1
369,A08161909WK3HU7UYTMW,B00000053B,5.0,2014-06-01,1
14212,A08161909WK3HU7UYTMW,B0000032XY,5.0,2014-01-12,1
40208,A08161909WK3HU7UYTMW,B0041WLBEC,4.0,2012-12-29,1
32108,A1020L7BWW9RAX,B0002X314C,4.0,2005-03-01,2


importing items

In [71]:
df_items = meta

In [72]:
df_items.head()

,asin,title,price,imUrl,related,salesRank,categories,description,brand
0,5555991584,Memory of Trees,9.49,http://ecx.images-amazon.com/images/I/51b5WDjd...,"{'also_bought': ['B000002LRT', 'B000002LRR', '...",{'Music': 939190},"[[CDs & Vinyl, New Age, Celtic New Age], [CDs ...",NaN,NaN
1,6308051551,Don't Drink His Blood,8.91,http://ecx.images-amazon.com/images/I/31LT2n7Q...,NaN,NaN,"[[Digital Music, Alternative Rock, Indie & Lo-...",NEW Combo BLUWAVS CD and FLAC FILE,NaN
2,7901622466,On Fire,11.33,http://ecx.images-amazon.com/images/I/21W29WZw...,"{'also_bought': ['B00000282A', 'B0084O8O9S', '...",{'Music': 58799},"[[CDs & Vinyl, Christian, Rock & Alternative],...",NaN,NaN
3,B0000000ZW,Changing Faces,23.64,http://ecx.images-amazon.com/images/I/51H3Bp-3...,"{'also_bought': ['B00000010Z', 'B0000039Q5', '...",{'Music': 68784},"[[CDs & Vinyl, Dance & Electronic], [CDs & Vin...",,
4,B00000016W,Pet Sounds,9.49,http://ecx.images-amazon.com/images/I/51pDGkXj...,"{'also_bought': ['B0025KVLTM', 'B00007FOMP', '...",{'Music': 77205},"[[CDs & Vinyl, Classic Rock], [CDs & Vinyl, Po...",NaN,NaN


In [73]:
# extracting specific columns
df_items = df_items[["asin", "title", "description", "imUrl"]]
df_items[["old_id"]] = df_items[['asin']]
# renaming columns
df_items = df_items.rename(columns={'asin': 'id', 'title': 'title', 'description': 'description', 'imUrl': 'imgurl'})


df_items = pd.merge(left=df_items,right=item_filter, left_on='id', right_on='item_id')


C:\Users\Nabeel\Anaconda2\envs\py36\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [74]:
df_items.head()

,id,title,description,imgurl,old_id,item_id
0,5555991584,Memory of Trees,NaN,http://ecx.images-amazon.com/images/I/51b5WDjd...,5555991584,5555991584
1,B00000016W,Pet Sounds,NaN,http://ecx.images-amazon.com/images/I/51pDGkXj...,B00000016W,B00000016W
2,B00000016T,Cars,NaN,http://ecx.images-amazon.com/images/I/51pmW%2B...,B00000016T,B00000016T
3,B00000017R,Jazz Samba,NaN,http://ecx.images-amazon.com/images/I/51kAU0Ty...,B00000017R,B00000017R
4,B0000004VW,Greatest Hits,,http://ecx.images-amazon.com/images/I/511vfCwh...,B0000004VW,B0000004VW


In [75]:
# creating keys for items
df_items = df_items.sort_values("id")
df_items['item_key'] = range(len(df_items))

df_items = df_items[["item_key", "title", "description", "imgurl","old_id"]]
df_items = df_items.rename(columns={'item_key': 'id'})

df_items = df_items.sort_values("id")
df_items = df_items.iloc[1:]


df_items_key = df_items[["old_id", "id"]]
df_items_key = df_items_key.rename(columns={'id': 'item_key'})

In [76]:
df_items.head()

,id,title,description,imgurl,old_id
2,1,Cars,NaN,http://ecx.images-amazon.com/images/I/51pmW%2B...,B00000016T
1,2,Pet Sounds,NaN,http://ecx.images-amazon.com/images/I/51pDGkXj...,B00000016W
3,3,Jazz Samba,NaN,http://ecx.images-amazon.com/images/I/51kAU0Ty...,B00000017R
5,4,By All Means Necessary,"After Scott La Rock's death, KRS-One (a.k.a. K...",http://ecx.images-amazon.com/images/I/41B2F7KN...,B0000004UM
4,5,Greatest Hits,,http://ecx.images-amazon.com/images/I/511vfCwh...,B0000004VW


Now, combining columns with the ratings

In [77]:
df_ratings = pd.merge(df_ratings, df_items_key, left_on = "item_id", right_on="old_id")

df_ratings = df_ratings[["userid_key", "item_key", "rating", 'timestamp', "user_id"]]
df_ratings = df_ratings.rename(columns={'user_id': 'old_user_id'})

df_ratings = df_ratings.rename(columns={'userid_key': 'user_id', 'item_key': 'item_id'})

df_ratings = df_ratings.sort_values("user_id")

Final format of ratings

In [78]:
df_ratings.head()

,user_id,item_id,rating,timestamp,old_user_id
0,1,206,3.0,2013-01-18,A08161909WK3HU7UYTMW
39,1,9,5.0,2014-06-01,A08161909WK3HU7UYTMW
92,1,940,4.0,2012-12-29,A08161909WK3HU7UYTMW
63,1,355,5.0,2014-01-12,A08161909WK3HU7UYTMW
331,2,757,4.0,2006-11-08,A1020L7BWW9RAX


In [79]:
print(df_ratings.shape)
print(df_items.shape)

(42094, 5)
(999, 5)


In [80]:
df_ratings.to_csv("cleaned_ratings.csv", index=False)
df_items.to_csv("cleaned_items.csv", index=False)